In [1]:
!pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek


In [4]:
df = pd.read_csv("Creditcard_data.csv")
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df['Class'].value_counts()


,count
Class,
0,763
1,9


In [6]:
X = df.drop('Class', axis=1)
y = df['Class']


In [7]:
ros = RandomOverSampler(random_state=42)
X_bal, y_bal = ros.fit_resample(X, y)


In [7]:
pd.Series(y_bal).value_counts()


In [8]:
samples = []

for i in range(5):
    X_s, _, y_s, _ = train_test_split(
        X_bal, y_bal,
        train_size=0.6,
        random_state=i
    )
    samples.append((X_s, y_s))


In [9]:
samplers = {
    "Sampling1": None,
    "Sampling2": RandomOverSampler(random_state=42),
    "Sampling3": RandomUnderSampler(random_state=42),
    "Sampling4": SMOTE(random_state=42),
    "Sampling5": SMOTETomek(random_state=42)
}


In [10]:
models = {
    "M1": LogisticRegression(max_iter=1000),
    "M2": DecisionTreeClassifier(),
    "M3": RandomForestClassifier(),
    "M4": GaussianNB(),
    "M5": SVC()
}


In [11]:
results = pd.DataFrame(index=models.keys(), columns=samplers.keys())


In [12]:
for m_name, model in models.items():
    for s_name, sampler in samplers.items():

        X_s, y_s = samples[0]   # using sample1 for consistency

        if sampler is not None:
            X_res, y_res = sampler.fit_resample(X_s, y_s)
        else:
            X_res, y_res = X_s, y_s

        X_train, X_test, y_train, y_test = train_test_split(
            X_res, y_res, test_size=0.3, random_state=42
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred) * 100
        results.loc[m_name, s_name] = round(acc, 2)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [13]:
results


,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
M1,93.45,93.53,94.12,93.17,93.17
M2,97.82,97.48,99.26,97.84,98.2
M3,99.64,99.64,100.0,99.64,99.64
M4,81.45,75.9,80.15,75.54,75.54
M5,65.45,65.47,69.12,65.47,65.47


In [15]:
results.index = [
    "Logistic Regression",
    "Decision Tree",
    "Random Forest",
    "Naive Bayes",
    "Support Vector Machine"
]


In [16]:
results.idxmax(axis=1)


,0
Logistic Regression,Sampling3
Decision Tree,Sampling3
Random Forest,Sampling3
Naive Bayes,Sampling1
Support Vector Machine,Sampling3
